In [3]:
# !pip uninstall -y torch 
# !pip install -U torch==2.0.0 
# !pip install -U transformers==4.33 
# !pip install accelerate==0.23
# !pip install typing_extensions

In [10]:
import pandas as pd
import torch

from settings import *
import os 
from transformers import pipeline, AutoModelForCausalLM

In [98]:
MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    cache_dir=os.path.join(BASE_DIR),
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to('cuda:0')

In [112]:
import re 
pipe = pipeline(
    'text-generation', 
    model=model,
    tokenizer=MODEL, 
    device=0
)

def ask(x, sbj_entity, obj_entity):
    ans = pipe(
        f"#### 질문:\n 문장'{re.sub('^[가-힣ㄱ-ㅎㅏ-ㅣ ]', '', x)}'에서 {sbj_entity}랑 {obj_entity}는 수정하지 말고, 기존 문장을 최대한 유지하면서, 유사한 문장을 만들어줘." + "\n\n #### 답변: \n", 
        do_sample=True, 
        max_new_tokens=256,
        temperature=0.5,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2
    )
    return ans[0]['generated_text']

In [113]:
train_set = pd.read_csv(os.path.join(DATA_DIR, 'train-v.0.0.2.csv'))

In [114]:
from ast import literal_eval
train_set.loc[:, 'subject_entity'] = train_set.loc[:, 'subject_entity'].apply(lambda x: literal_eval(x)['word'])
train_set.loc[:, 'object_entity'] = train_set.loc[:, 'object_entity'].apply(lambda x: literal_eval(x)['word'])

In [115]:
for idx, rows in train_set.iterrows():
    x, sbj, obj, label = rows['sentence'], rows['subject_entity'], rows['object_entity'], rows['label']
    outs = ask(x, sbj, obj) 
    print('subject entity')
    print(f'Origin Sentence: {x}\n')
    print(f'Generate Sentence: {outs}')
    break 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


subject entity
Origin Sentence: 추승우(秋承佑, 1979년 9월 24일 ~)는 전 KBO 리그 한화 이글스의 외야수이자, 현재 KBO 리그 한화 이글스의 작전·주루 및 외야수 수비 코치이다.

Generate Sentence: "가을에는 야구의 맛이 최고입니다. 선수들이 높은 하늘과 푸른 바다를 배경으로 플레이하는 모습이 정말 아름답습니다. 또한, 그라운드에서는 다양한 스포츠가 이루어지며, 야구를 즐기는 것이 인생에서 가장 좋은 경험 중 하나입니다."
